<a href="https://colab.research.google.com/github/mumumu99/L0-norm-Problem/blob/main/L0_norm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
import wget, pickle, os

def load_data():
    # Load dataset from DIYA GitLab
    url = "https://gitlab.diyaml.com/moong1234/application/raw/release/data.pkl"
    if not os.path.isfile("data.pkl"):
        wget.download(url, 'data.pkl')
    
    path = "data.pkl"
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

data, label = load_data()
print(data.shape, label.shape)
data, label

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=2e999de566fad2c2af0097e8e259b0c90686f96dee29b11451b227d2689af4dc
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
(4096, 12) (4096, 2)


(array([[ 1.,  1.,  1., ...,  1., -1.,  1.],
        [ 1., -1., -1., ..., -1.,  1.,  1.],
        [-1.,  1.,  1., ..., -1., -1., -1.],
        ...,
        [ 1., -1.,  1., ...,  1.,  1.,  1.],
        [ 1.,  1.,  1., ..., -1., -1.,  1.],
        [-1.,  1., -1., ...,  1.,  1., -1.]]), array([[0., 1.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

In [ ]:
import torch
from torch import nn, optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ratio = [.6, .2, .2]

train_data_x = data[:int(data.shape[0]*ratio[0]), :]
test_data_x = data[int(data.shape[0]*ratio[0]):int(data.shape[0]*(1-ratio[1])), :]
validation_data_x = data[int(data.shape[0]*(1-ratio[1])):, :]

train_data_y = label[:int(data.shape[0]*ratio[0]), :]
test_data_y = label[int(data.shape[0]*ratio[0]):int(data.shape[0]*(1-ratio[1])), :]
validation_data_y = label[int(data.shape[0]*(1-ratio[1])):, :]

train_data_x = torch.tensor(train_data_x, dtype=torch.float64, device=device).double()
train_data_y = torch.tensor(train_data_y, dtype=torch.float64, device=device).double()
test_data_x = torch.tensor(test_data_x, dtype=torch.float64, device=device).double()
test_data_y = torch.tensor(test_data_y, dtype=torch.float64, device=device).double()
validation_data_x = torch.tensor(validation_data_x, dtype=torch.float64, device=device).double()
validation_data_y = torch.tensor(validation_data_y, dtype=torch.float64, device=device).double()

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(train_data_x, train_data_y)
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()

        self.l1 = torch.nn.Linear(input_dim, hidden_dim)
        self.sigmoid = torch.nn.Sigmoid()
        self.l2 = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, x):
        out1 = self.sigmoid(self.l1(x))
        y_pred = self.sigmoid(self.l2(out1))

        return y_pred

model = MLP(12, 20, 2)

from torchsummary import summary as summary_
summary_(model, (2457, 12), batch_size=10)

In [ ]:
criterion = torch.nn.BCELoss(reduction = 'mean')
optimizer = optim.SGD(model.parameters(), lr = 5)

for epoch in range(10001):
    y_pred = model(train_data_x.float())
    loss = criterion(y_pred, train_data_y.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 200 == 0:
        print(f'Epoch : {epoch} / 10000 | Loss : {loss.item():.4f}')

Epoch : 0 / 10000 | Loss : 0.7050
Epoch : 200 / 10000 | Loss : 0.6918
Epoch : 400 / 10000 | Loss : 0.6909
Epoch : 600 / 10000 | Loss : 0.6876
Epoch : 800 / 10000 | Loss : 0.6836
Epoch : 1000 / 10000 | Loss : 0.6807
Epoch : 1200 / 10000 | Loss : 0.6850
Epoch : 1400 / 10000 | Loss : 0.6799
Epoch : 1600 / 10000 | Loss : 0.6784
Epoch : 1800 / 10000 | Loss : 0.6763
Epoch : 2000 / 10000 | Loss : 0.6186
Epoch : 2200 / 10000 | Loss : 0.4182
Epoch : 2400 / 10000 | Loss : 0.2115
Epoch : 2600 / 10000 | Loss : 0.1500
Epoch : 2800 / 10000 | Loss : 0.1098
Epoch : 3000 / 10000 | Loss : 0.0799
Epoch : 3200 / 10000 | Loss : 0.0631
Epoch : 3400 / 10000 | Loss : 0.0521
Epoch : 3600 / 10000 | Loss : 0.0441
Epoch : 3800 / 10000 | Loss : 0.0378
Epoch : 4000 / 10000 | Loss : 0.0327
Epoch : 4200 / 10000 | Loss : 0.0286
Epoch : 4400 / 10000 | Loss : 0.0252
Epoch : 4600 / 10000 | Loss : 0.0224
Epoch : 4800 / 10000 | Loss : 0.0202
Epoch : 5000 / 10000 | Loss : 0.0183
Epoch : 5200 / 10000 | Loss : 0.0167
Epoch : 